In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from scipy.stats import norm
from tqdm.auto import tqdm

import sys
sys.path.append('../utils')
from utils import *

99.88102321451903


C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
combined_df = retrieve_data()
combined_df.head()

,Date,ROG_Last_Price,ROG_IVOL,CFR_Last_Price,CFR_IVOL,ZURN_Last_Price,ZURN_IVOL,Risk_Free_Rate
0,2023-10-25,237.55,17.6858,107.00,33.0793,426.4,15.7900,1.4594
1,2023-10-26,237.25,17.9504,104.45,33.5406,426.8,15.9736,1.4508
2,2023-10-27,237.75,19.6028,104.40,33.4355,424.4,16.3945,1.4478
3,2023-10-30,241.30,19.3503,104.75,33.1531,428.2,15.9477,1.4623
4,2023-10-31,233.85,19.2518,106.90,32.7788,430.6,15.7948,1.4489


In [4]:
np.random.seed(42)
n_days = 252
dt = 1 / n_days

close = combined_df[['ROG_Last_Price', 'CFR_Last_Price', 'ZURN_Last_Price']]
close.cov()

,ROG_Last_Price,CFR_Last_Price,ZURN_Last_Price
ROG_Last_Price,375.344673,-17.563543,286.937174
CFR_Last_Price,-17.563543,133.176550,138.191785
ZURN_Last_Price,286.937174,138.191785,706.118284


In [5]:
tickers = close.columns
log_returns = np.log((close / close.shift(1)).dropna())

In [ ]:
# Simulating from start


def MultivariateGBMSimulation(
    s0=close.iloc[0], 
    tickers=tickers,
    dt=dt,
    drift=combined_df['Risk_Free_Rate'].values / 100,
    volatility=log_returns.cov() * n_days,
    n_paths=100,
    T=1, # Time horizon in year:
    variance_reduction=None
    ):
    
    n_steps = int(T / dt)
    result = np.zeros((len(tickers), n_paths, n_steps))

    if variance_reduction==None:
        for i in tqdm(range(n_paths)):
            choleskyMatrix = np.linalg.cholesky(volatility)
            e = np.random.normal(size=(len(tickers), n_steps)) # Generate RV for steps


            for j in range(n_steps):
                for k in range(len(tickers)):
                    if(j==0):
                        result[k, i, j] = s0[tickers[k]]
                    else:
                        if isinstance(drift, np.ndarray):
                            result[k, i, j] = result[k, i, j-1] * np.exp(
                                (drift[j] -  1/2 * np.sqrt(volatility.iloc[k, k])) * dt + 
                                np.sqrt(dt) * choleskyMatrix[k, k] * e[k, j])
                        else:
                            result[k, i, j] = result[k, i, j-1] * np.exp(
                                (drift -  1/2 * np.sqrt(volatility.iloc[k, k])) * dt + 
                                np.sqrt(dt) * choleskyMatrix[k, k] * e[k, j])
    
    elif variance_reduction=='av':
        for i in tqdm(range(n_paths // 2)):
            choleskyMatrix = np.linalg.cholesky(volatility)
            e = np.random.normal(size=(len(tickers), n_steps)) # Generate RV for steps
            e_tilde = -e    

            for j in range(n_steps):
                for k in range(len(tickers)):
                    if(j==0):
                        result[k, i, j] = s0[tickers[k]]
                        result[k, n_paths - i - 1, j] = s0[tickers[k]]

                    else:
                        if isinstance(drift, np.ndarray):
                            result[k, i, j] = result[k, i, j-1] * np.exp(
                                (drift[j] -  1/2 * np.sqrt(volatility.iloc[k, k])) * dt + 
                                np.sqrt(dt) * choleskyMatrix[k, k] * e[k, j])
                            result[k, n_paths - i - 1, j] = result[k, n_paths - i - 1, j-1] * np.exp(
                                (drift[j] -  1/2 * np.sqrt(volatility.iloc[k, k])) * dt + 
                                np.sqrt(dt) * choleskyMatrix[k, k] * e_tilde[k, j])
                        else:
                            result[k, i, j] = result[k, i, j-1] * np.exp(
                                (drift -  1/2 * np.sqrt(volatility.iloc[k, k])) * dt + 
                                np.sqrt(dt) * choleskyMatrix[k, k] * e[k, j])
                            result[k, n_paths - i - 1, j] = result[k, n_paths - i - 1, j-1] * np.exp(
                                (drift -  1/2 * np.sqrt(volatility.iloc[k, k])) * dt + 
                                np.sqrt(dt) * choleskyMatrix[k, k] * e_tilde[k, j])
                            
    elif variance_reduction=='emc':
        for i in tqdm(range(n_paths)):
            choleskyMatrix = np.linalg.cholesky(volatility)
            e = np.random.normal(size=(len(tickers), n_steps)) # Generate RV for steps

            for j in range(n_steps):
                for k in range(len(tickers)):
                    if(j==0):
                        result[k, i, j] = s0[tickers[k]]
                    else:
                        if isinstance(drift, np.ndarray):
                            result[k, i, j] = result[k, i, j-1] * np.exp(
                                (drift[j] -  1/2 * np.sqrt(volatility.iloc[k, k])) * dt + 
                                np.sqrt(dt) * choleskyMatrix[k, k] * e[k, j])
                        else:
                            result[k, i, j] = result[k, i, j-1] * np.exp(
                                (drift -  1/2 * np.sqrt(volatility.iloc[k, k])) * dt + 
                                np.sqrt(dt) * choleskyMatrix[k, k] * e[k, j])
        for k in range(len(tickers)):  
                # path, step
            correction_factor = result[k][-1, :].mean() / result[k][-1, :] 
            result[k] = result[k] * correction_factor                    
           
    return result, tickers

In [ ]:
result_emc, _ = MultivariateGBMSimulation(s0=close.iloc[187], n_paths=1000, T=dt * 65, volatility=log_returns.cov() * n_days, variance_reduction='emc')

result, _ = MultivariateGBMSimulation(s0=close.iloc[187], n_paths=1000, T=dt * 65, volatility=log_returns.cov() * n_days, variance_reduction=None)

result_av, _ = MultivariateGBMSimulation(s0=close.iloc[187], n_paths=1000, T=dt * 65, volatility=log_returns.cov() * n_days, variance_reduction='av')

100%|██████████| 500/500 [00:03<00:00, 130.54it/s]


In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(8, 10))

for i in range(100):
    axes[0].plot(result_av[0][i], lw=1)
axes[0].set_title('Simulated Asset Price Paths for Roche (25/07/2024 - 25/10/2024)')
axes[0].set_xlabel('Days')
axes[0].set_ylabel('Asset Price')

for i in range(100):
    axes[1].plot(result_av[1][i], lw=1)
axes[1].set_title('Simulated Asset Price Paths for Richemont (25/07/2024 - 25/10/2024)')
axes[1].set_xlabel('Days')
axes[1].set_ylabel('Asset Price')

for i in range(100):
    axes[2].plot(result_av[2][i], lw=1)
axes[2].set_title('Simulated Asset Price Paths for Zurich (25/07/2024 - 25/10/2024)')
axes[2].set_xlabel('Days')
axes[2].set_ylabel('Asset Price')

plt.tight_layout()
plt.show()

In [ ]:
payoff = []
payoff_av = []
payoff_emc = []
for i in range(result.shape[1]):
    payoff.append(payoff_func(result[0][i], result[1][i], result[2][i]))
    payoff_av.append(payoff_func(result_av[0][i], result_av[1][i], result_av[2][i]))
    payoff_emc.append(payoff_func(result_emc[0][i], result_emc[1][i], result_emc[2][i]))

In [18]:
np.mean(payoff), np.std(payoff)

(1087.5, 0.0)

In [19]:
np.mean(payoff_av), np.std(payoff_av)

(1087.5, 0.0)

In [20]:
np.mean(payoff_emc), np.std(payoff_emc)

(1087.5, 0.0)